# TWITCH GAMERS NETWORK
Loris Palmarin (32349A)

Importing packages needed:

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
from pandas import isnull
from pandas.io.pytables import dropna_doc
from sklearn.preprocessing import StandardScaler

## Preprocessing

In [3]:
edgelist = pd.read_csv('large_twitch_edges.csv')
edgelist

,numeric_id_1,numeric_id_2
0,98343,141493
1,98343,58736
2,98343,140703
3,98343,151401
4,98343,157118
...,...,...
6797552,97507,29359
6797553,71175,12020
6797554,151702,128281
6797555,118034,38021


Rename columns in "source" and "target"

In [4]:
edgelist = edgelist.rename(columns={'numeric_id_1':'source', 'numeric_id_2':'target'})
edgelist

,source,target
0,98343,141493
1,98343,58736
2,98343,140703
3,98343,151401
4,98343,157118
...,...,...
6797552,97507,29359
6797553,71175,12020
6797554,151702,128281
6797555,118034,38021


In [7]:
edgelist.isnull().sum()

source    0
target    0
dtype: int64

Create graph:

In [13]:
G = nx.from_pandas_edgelist(edgelist, source='source', target='target')
print(G)
print('Is the graph directed ?',G.is_directed())

Graph with 168114 nodes and 6797557 edges
Is the graph directed ? False


Upload note features

In [19]:
node_features = pd.read_csv("large_twitch_features.csv")
node_features

,views,mature,life_time,created_at,updated_at,numeric_id,dead_account,language,affiliate
0,7879,1,969,2016-02-16,2018-10-12,0,0,EN,1
1,500,0,2699,2011-05-19,2018-10-08,1,0,EN,0
2,382502,1,3149,2010-02-27,2018-10-12,2,0,EN,1
3,386,0,1344,2015-01-26,2018-10-01,3,0,EN,0
4,2486,0,1784,2013-11-22,2018-10-11,4,0,EN,0
...,...,...,...,...,...,...,...,...,...
168109,4965,0,810,2016-07-20,2018-10-08,168109,0,EN,0
168110,4128,1,2080,2013-01-31,2018-10-12,168110,0,EN,0
168111,3545,0,1797,2013-11-08,2018-10-10,168111,0,EN,1
168112,892736,1,2135,2012-12-07,2018-10-12,168112,0,EN,0


Add features to nodes in graph G

In [20]:
# set ID
node_features.set_index("numeric_id", inplace=True)

for node, features in node_features.iterrows():
    G.nodes[node]["features"] = features.to_dict()

Check correctness of application

In [26]:
example_node = list(G.nodes)[0]
print(f"Feature per il nodo {example_node}: {G.nodes[example_node].get('features', {})}")



Feature per il nodo 98343: {'views': 282, 'mature': 0, 'life_time': 2086, 'created_at': '2012-12-27', 'updated_at': '2018-09-13', 'dead_account': 0, 'language': 'EN', 'affiliate': 0}
